# Coordinate systems and transforms of Synthesis AI face dataset

At first, we install some dependencies.

Installing `face_api_dataset` from the github it it's not already installed.

In [1]:
%%bash

(python -c "import face_api_dataset")
if [ $? -eq 0 ]
then
    echo "face_api_dataset is already installed"
else
    echo "installing face_api_dataset from github"
    pip install git+https://github.com/Synthesis-AI-Dev/face_api_dataset.git
fi

face_api_dataset is already installed


Installing `git lfs` from the github it it's not already installed.

In [2]:
%%bash

(git lfs install > /dev/null)
if [ $? -eq 0 ]
then
    echo "git lfs is already installed"
else
    echo "installing git lfs"
    case "$(uname -s)" in

       Darwin)
         brew install git-lfs
         ;;

       Linux)
         curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.deb.sh | sudo bash
         apt-get install git-lfs
         ;;

       *)
         echo "This cell can't install git lfs your OS. Please take care of it yourself."
         ;;
    esac
fi

git lfs is already installed


Pulling the test dataset for the example

In [3]:
%%bash

if [ -d "../test_dataset" ]
then
    echo "test_dataset is already present"
else
    echo "downloading test_dataset"
    git clone https://github.com/Synthesis-AI-Dev/face_api_dataset
    mv face_api_dataset/test_dataset ..
    rm -rf face_api_dataset
fi

test_dataset is already present


Finally we add some imports for the visualisation.

In [4]:
%matplotlib inline

In [5]:
import numpy as np

We use `FaceApiDataset` class to access synthesis datasets.


In [6]:
from face_api_dataset import FaceApiDataset, Modality

**Warning!** Some of modalities requires additional libraries to be installed:
`SEGMENTS` and `RGB` modalities use `opencv-python` library,
 while `DEPTH`, `ALPHA` and `NORMALS` modalities
 use `tiffile`  and `imagecodecs` libraries for effective work with floating point tiff files.
 If dataset with these modalities will be created without corresponding libraries present, an `ImportError` is raised.


In [7]:
data_root = "../test_dataset"
dataset = FaceApiDataset(data_root,
                        modalities=[Modality.RGB, Modality.CAM_TO_HEAD, Modality.HEAD_TO_CAM,
                                    Modality.WORLD_TO_HEAD, Modality.HEAD_TO_WORLD, Modality.WORLD_TO_CAM,
                                    Modality.CAM_TO_WORLD, Modality.LANDMARKS_3D_IBUG68
                                   ])

In [8]:
len(dataset)

13

There are 13 items in the test dataset. Let's explore them closer.

In [9]:
item = dataset[0]
item2 = dataset[1]

Each item is a dict with different modalities as keys.

In [10]:
print(item.keys())

dict_keys([<Modality.RGB: 2>, <Modality.LANDMARKS_3D_IBUG68: 13>, <Modality.HEAD_TO_CAM: 27>, <Modality.CAM_TO_HEAD: 28>, <Modality.HEAD_TO_WORLD: 29>, <Modality.WORLD_TO_HEAD: 30>, <Modality.CAM_TO_WORLD: 31>, <Modality.WORLD_TO_CAM: 32>])


In Synthesis AI face dataset we use 3 different coordinate systems:
1.  World coordinate system.
2.  Camera coordinate system, centered at the camera with x-axis directed right, y-axis directed top and z-axis directed towards the camera.
3.  Face coordinate system, centered at the center of the face with x-axis directed toward the left ear, y-axis directe to the top of the head and z-axis directed towards the noze.

We provide transformations between head,camera and world coordinates systems as 4x4 matrices in homogeneous coordinates.

In [11]:
item[Modality.CAM_TO_HEAD]

array([[ 0.93239178,  0.02019352,  0.36088381,  0.38612463],
       [ 0.06352884,  0.97373945, -0.21862133, -0.23481784],
       [-0.35582242,  0.22676728,  0.90662398,  0.91507426],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [12]:
item[Modality.HEAD_TO_CAM]

array([[ 0.93239212,  0.06352874, -0.3558217 , -0.0194986 ],
       [ 0.0201936 ,  0.97373974,  0.22676736,  0.01334524],
       [ 0.36088476, -0.21862136,  0.9066242 , -1.02031116],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [13]:
item[Modality.HEAD_TO_WORLD]

array([[ 0.93239212,  0.06352874, -0.3558217 , -0.01929043],
       [ 0.0201936 ,  0.97373974,  0.22676736,  1.55267489],
       [ 0.36088476, -0.21862136,  0.9066242 ,  0.04655426],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [14]:
item[Modality.WORLD_TO_HEAD]

array([[ 0.93239178,  0.02019352,  0.36088381, -0.03016841],
       [ 0.06352884,  0.97373945, -0.21862133, -1.50049754],
       [-0.35582242,  0.22676728,  0.90662398, -0.40116704],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [15]:
item[Modality.CAM_TO_WORLD]

array([[ 1.00000000e+00, -1.16382334e-18,  1.40946303e-18,
         2.08167286e-04],
       [ 1.16382334e-18,  1.00000000e+00,  6.58739521e-17,
         1.53932965e+00],
       [-1.40946303e-18, -6.58739521e-17,  1.00000000e+00,
         1.06686541e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00]])

In [16]:
item[Modality.WORLD_TO_CAM]

array([[ 1.00000000e+00,  1.16551734e-18, -1.40946303e-18,
        -2.08167286e-04],
       [-1.16382334e-18,  1.00000000e+00, -6.58739521e-17,
        -1.53932965e+00],
       [ 1.40946303e-18,  1.11022302e-16,  1.00000000e+00,
        -1.06686541e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00]])

X_TO_Y transform is always inverse of Y_TO_X:

In [17]:
np.testing.assert_allclose(item[Modality.CAM_TO_HEAD] @ item[Modality.HEAD_TO_CAM], np.eye(4), atol=1e-10)
np.testing.assert_allclose(item[Modality.HEAD_TO_WORLD] @ item[Modality.WORLD_TO_HEAD], np.eye(4), atol=1e-10)
np.testing.assert_allclose(item[Modality.CAM_TO_WORLD] @ item[Modality.WORLD_TO_CAM], np.eye(4), atol=1e-10)

And composition of X_TO_Y and Z_TO_X transforms always give Z_TO_Y transform:

In [18]:
np.testing.assert_allclose(item[Modality.CAM_TO_WORLD], item[Modality.HEAD_TO_WORLD] @ item[Modality.CAM_TO_HEAD], atol=1e-10)


All 3D coordinates (ex. landmarks) are always given in camera coordinates.

In [21]:
item[Modality.LANDMARKS_3D_IBUG68]['0']

(-0.07423346489667892, 0.0001043081283569336, -1.0857646465301514)

If we want to get coordinates in the face coordinates (they should be independent on camera position and pitch/yaw/roll values),
we need to multiply them by CAM_TO_HEAD transform.

However, as matrices are in homogeneous coordinates, first we will need to transform them to homogeneous coordinates:

In [35]:
def to_homogeneous(xs):
    xs_array = np.array(xs)
    assert (xs_array.shape[-1] == 3)
    ones_shape = list(xs_array.shape)
    ones_shape[-1] = 1
    homogeneous = np.concatenate([xs_array, np.ones(ones_shape)], axis=-1)
    return homogeneous

In [36]:
homogeneous_cam = to_homogeneous(item[Modality.LANDMARKS_3D_IBUG68]['0'])
homogeneous_cam

TypeError: 'tuple' object does not support item assignment

Now, we can compute homogeneous coordinates in head coordinate system.

In [30]:
homogeneous_head = item[Modality.CAM_TO_HEAD] @ homogeneous_cam
homogeneous_head

array([-0.07492283, -0.00206093, -0.04286842,  1.        ])

Now, we need to transform back to euclidian coordinates:

In [39]:
def from_homogeneous(xs):
    xs_array = np.array(xs)
    assert (xs_array.shape[-1] == 4)

    slice_but_last = list(xs_array.shape)
    slice_but_last[-1] = slice(None, -1)
    slice_but_last = tuple(slice_but_last)

    slice_last = list(xs_array.shape)
    slice_last[-1] = slice(-1, None)
    slice_last = tuple(slice_last)

    return xs_array[slice_but_last] / xs_array[slice_last]

And we can compute euclidian coordinates in the head space as:

In [41]:
from_homogeneous(homogeneous_head)

array([-0.07492283, -0.00206093, -0.04286842])